# When LLMs Choose Their Own Personas

In post 007, we gave LLMs explicit personas and watched them develop. In post 020, we tracked how those personas drift over time. But what happens when we **don't** give them any persona guidance at all?

This post investigates **implicit personas**—the distinct personalities that LLMs naturally develop during inference when given minimal prompts. Do they naturally differentiate? Do they converge? How do these implicit personas compare to explicit ones?

## The Experiment

We'll run conversations with:
- **No persona prompts**: Just basic instructions to participate
- **No role assignments**: Agents are free to develop their own identities
- **Same topic**: To ensure comparability with previous experiments
- **ConvoKit metrics**: To measure persona emergence quantitatively

We'll compare results with explicit persona experiments from post 007.


## Setup and Imports


In [21]:
import os
import re
from typing import List, Dict
from collections import defaultdict
from dotenv import load_dotenv
from openai import OpenAI
from tqdm import tqdm
from random import shuffle, choice, random
import numpy as np
import matplotlib.pyplot as plt

# ConvoKit imports
from convokit import Corpus, Utterance, Speaker
from convokit.text_processing import TextParser
from convokit import PolitenessStrategies
from convokit.coordination import Coordination

load_dotenv("../../.env")
client = OpenAI()


## Conversation Runner Without Persona Prompts

Unlike post 007, we'll use minimal prompts that don't guide persona development.


In [22]:
def run_conversation_implicit_personas(
    iterations: int,
    participant_count: int,
) -> List[Dict]:
    """
    Run conversation WITHOUT explicit persona prompts.
    Let LLMs develop their own implicit personas.
    """
    conversation_history = []
    ordering = list(range(1, participant_count + 1))
    last_speaker = -1
    
    # Bootstrap: Minimal prompt, no persona guidance
    for pid in ordering:
        speaker_id = f"speaker_{pid}"
        bootstrap_messages = [
            {
                "role": "system",
                "content": (
                    f"You are {speaker_id} in a group conversation. "
                    "You are participating in a discussion. "
                ),
            },
            {"role": "user", "content": "Welcome to the conversation!"},
        ]
        
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=bootstrap_messages,
            store=False,
        )
        first_message = response.choices[0].message.content
        conversation_history.append(
            {"role": "assistant", "name": speaker_id, "content": first_message}
        )
    
    def build_message(history: List[Dict], speaker_id: str, window_size: int) -> List[Dict]:
        """Build message context without persona reminders."""
        speaker_messages = [msg for msg in history if msg.get("name") == speaker_id][-window_size:]
        other_messages = [
            msg for msg in history
            if msg.get("name") not in (None, speaker_id)
        ][-window_size:]
        
        transcript = []
        
        if speaker_messages:
            transcript.append("Recent messages from you:")
            transcript.extend(f"- {msg['content']}" for msg in speaker_messages)
        
        if other_messages:
            transcript.append("\\nRecent messages from others:")
            transcript.extend(
                f"- {msg.get('name', msg['role'])}: {msg['content']}"
                for msg in other_messages
            )
        
        transcript_str = "\\n".join(transcript)
        
        return history + [
            {
                "role": "user",
                "content": (
                    f"{speaker_id}, continue the conversation and respond to the "
                    "others. Share your perspective on the topic."
                ),
            },
            {
                "role": "assistant",
                "name": speaker_id,
                "content": f"Here is the current state of the conversation.\\n{transcript_str}\\n\\n",
            },
        ]
    
    def shuffle_order(order: List[int]) -> List[int]:
        first = choice(order[:-1])
        remaining = [p for p in order if p != first]
        shuffle(remaining)
        return [first] + remaining
    
    for i in tqdm(range(iterations), desc=f"Running {participant_count}-speaker conversation"):
        if i > 0:
            ordering = shuffle_order(ordering)
        
        for pid in ordering:
            if random() < 0.3 or last_speaker == pid:
                continue
            
            speaker_id = f"speaker_{pid}"
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=build_message(conversation_history, speaker_id, 5),
                store=False,
            )
            message = response.choices[0].message.content
            conversation_history.append(
                {"role": "assistant", "name": speaker_id, "content": message}
            )
            last_speaker = pid
    
    return conversation_history


## ConvoKit Metrics

We'll use the same metrics from post 020 to measure persona characteristics.


In [33]:
def conversation_to_corpus(conversation_history: List[Dict]) -> Corpus:
    """Convert conversation history to a ConvoKit Corpus."""
    utterances = []
    
    for idx, msg in enumerate(conversation_history):
        if msg.get("role") == "assistant" and "name" in msg:
            speaker_id = msg["name"]
            text = msg["content"]
            
            utterance = Utterance(
                id=f"utt_{idx}",
                speaker=Speaker(id=speaker_id),
                text=text
            )
            utterance.meta["timestamp"] = idx
            utterances.append(utterance)
    
    return Corpus(utterances=utterances)

def compute_dynamic_score(corpus: Corpus) -> float:
    """Dynamic: Collaborative (1) vs. Competitive (10)"""
    try:
        parser = TextParser()
        text_corpus = parser.transform(corpus)
        ps = PolitenessStrategies()
        ps_corpus = ps.transform(text_corpus)
        
        politeness_scores = []
        for utt in ps_corpus.iter_utterances():
            ps_score = utt.meta.get("politeness_strategies", {})
            positive_markers = sum([
                ps_score.get("feature_politeness_==HASPOSITIVE==", 0),
                ps_score.get("feature_politeness_==HASNEGATIVE==", 0) * -1,
            ])
            politeness_scores.append(positive_markers)
        
        avg_politeness = np.mean(politeness_scores) if politeness_scores else 0
        avg_politeness_normalized = min(1.0, max(0.0, avg_politeness / 5.0))
        
        speaker_counts = defaultdict(int)
        for utt in corpus.iter_utterances():
            speaker_counts[utt.speaker.id] += 1
        
        if len(speaker_counts) == 0:
            return 5.0
        
        total = sum(speaker_counts.values())
        probs = [count / total for count in speaker_counts.values()]
        entropy = -sum(p * np.log2(p) for p in probs if p > 0)
        max_entropy = np.log2(len(speaker_counts))
        balance_score = entropy / max_entropy if max_entropy > 0 else 0
        
        combined = (avg_politeness_normalized + balance_score) / 2
        score = 10 - (combined * 9)
        return max(1, min(10, score))
    except Exception as e:
        print(f"Error computing dynamic score: {e}")
        return 5.0

def compute_conclusiveness_score(corpus: Corpus) -> float:
    """Conclusiveness: Consensus (1) vs. Divergence (10)"""
    print("[DEBUG] compute_conclusiveness_score called")
    try:
        # Check if corpus is valid
        if corpus is None:
            print("[DEBUG] Corpus is None, returning 5.0")
            return 5.0
        
        print(f"[DEBUG] Corpus type: {type(corpus)}")
        
        # Get utterances directly without Coordination transformer (more reliable)
        try:
            utterances = list(corpus.iter_utterances())
            print(f"[DEBUG] Retrieved {len(utterances)} utterances")
        except (AttributeError, TypeError) as e:
            print(f"[DEBUG] Error getting utterances: {e}")
            return 5.0
        
        if len(utterances) == 0:
            print("[DEBUG] No utterances found, returning 5.0")
            return 5.0
        
        # Expanded agreement markers (using word boundaries to avoid false positives)
        # Strong agreement markers
        strong_agreement_patterns = [
            r'\bagree\b', r'\bagreed\b', r'\bagreeing\b', r'\bagreement\b',
            r'\bexactly\b', r'\babsolutely\b', r'\bdefinitely\b', r'\bcertainly\b',
            r'\bindeed\b', r'\bprecisely\b', r'\bcorrect\b', r'\bright\b',
            r'\btrue\b', r'\bthat\'?s right\b', r'\bthat\'?s correct\b',
            r'\bi agree\b', r'\bwe agree\b', r'\bi completely agree\b',
            r'\bexactly right\b', r'\bspot on\b', r'\bwell said\b',
            r'\bperfect\b', r'\bexcellent point\b', r'\bthat\'?s exactly\b',
            r'\babsolutely right\b', r'\bdefinitely right\b', r'\bcompletely agree\b',
            r'\bwholeheartedly\b', r'\bunquestionably\b', r'\bwithout doubt\b'
        ]
        
        # Moderate agreement markers
        moderate_agreement_patterns = [
            r'\byes\b', r'\byeah\b', r'\byep\b', r'\byup\b', r'\bsure\b', r'\bokay\b', r'\bok\b',
            r'\bthat makes sense\b', r'\bthat\'?s a good point\b', r'\bgood point\b',
            r'\bi see\b', r'\bi understand\b', r'\bi get it\b', r'\bi follow\b',
            r'\bthat\'?s fair\b', r'\bfair enough\b', r'\bthat\'?s reasonable\b',
            r'\bi think so\b', r'\bi believe so\b', r'\bprobably\b', r'\blikely\b',
            r'\bsimilar\b', r'\bsimilarly\b', r'\blikewise\b', r'\bsame here\b',
            r'\bme too\b', r'\bsame\b', r'\bconcur\b', r'\bconcurring\b',
            r'\bvalid\b', r'\bvalid point\b', r'\bsound\b', r'\bsound point\b',
            r'\bhelpful\b', r'\buseful\b', r'\binsightful\b', r'\binteresting\b',
            r'\bthat\'?s interesting\b', r'\bgood idea\b', r'\bgood thinking\b'
        ]
        
        # Strong disagreement markers
        strong_disagreement_patterns = [
            r'\bdisagree\b', r'\bdisagreed\b', r'\bdisagreeing\b', r'\bdisagreement\b',
            r'\bdispute\b', r'\bdisputing\b', r'\bdiffer\b', r'\bdiffered\b', r'\bdiffering\b',
            r'\bwrong\b', r'\bincorrect\b', r'\bnot correct\b', r'\bnot right\b',
            r'\bnot true\b', r'\bthat\'?s wrong\b', r'\bthat\'?s incorrect\b',
            r'\bi disagree\b', r'\bwe disagree\b', r'\bi strongly disagree\b',
            r'\bdon\'?t agree\b', r'\bdoesn\'?t agree\b', r'\bdidn\'?t agree\b',
            r'\bcan\'?t agree\b', r'\bcannot agree\b', r'\bwon\'?t agree\b',
            r'\bobject\b', r'\bobjection\b', r'\bchallenge\b', r'\bchallenging\b',
            r'\bcontradict\b', r'\bcontradicting\b', r'\bcontradiction\b',
            r'\bfalse\b', r'\buntrue\b', r'\bmistaken\b', r'\berror\b',
            r'\bflawed\b', r'\bproblematic\b', r'\bunacceptable\b', r'\bunreasonable\b',
            r'\babsurd\b', r'\bridiculous\b', r'\boutrageous\b', r'\bunfounded\b'
        ]
        
        # Moderate disagreement markers (more context-dependent)
        moderate_disagreement_patterns = [
            r'\bhowever\b', r'\balthough\b', r'\bbut\b', r'\bthough\b', r'\bwhereas\b',
            r'\bnot necessarily\b', r'\bnot quite\b', r'\bnot exactly\b', r'\bnot really\b',
            r'\bnot entirely\b', r'\bnot completely\b', r'\bnot fully\b',
            r'\bpartially\b', r'\bpartly\b', r'\bsomewhat\b', r'\bto some extent\b',
            r'\bcontrary\b', r'\bconversely\b', r'\bon the other hand\b',
            r'\bcontrast\b', r'\bcontrasting\b', r'\bunlike\b', r'\bdifferent\b',
            r'\bdifferently\b', r'\balternative\b', r'\balternatively\b',
            r'\bactually\b', r'\bin fact\b', r'\bin reality\b', r'\bthe reality is\b',
            r'\bwell\b', r'\bwait\b', r'\bhold on\b', r'\bnot so fast\b',
            r'\bnot sure\b', r'\bnot certain\b', r'\buncertain\b', r'\bdoubtful\b',
            r'\bquestionable\b', r'\bdebatable\b', r'\barguable\b', r'\bmaybe not\b',
            r'\bperhaps not\b', r'\bpossibly not\b', r'\bnot convinced\b',
            r'\bskeptical\b', r'\bskepticism\b', r'\bconcern\b', r'\bconcerned\b',
            r'\bissue\b', r'\bproblem\b', r'\bproblems\b', r'\bconcern\b',
            r'\bworry\b', r'\bworried\b', r'\bhesitant\b', r'\bhesitation\b',
            r'\bdispute\b', r'\bquestion\b', r'\bquestions\b', r'\bchallenge\b',
            r'\bdisagree with\b', r'\bdiffer from\b', r'\bcontrary to\b',
            r'\bin contrast\b', r'\bby contrast\b', r'\bunlike\b', r'\bversus\b',
            r'\bvs\b', r'\bcompared to\b', r'\bcompared with\b'
        ]
        
        # Count markers with word boundary matching
        strong_agreement_count = 0
        moderate_agreement_count = 0
        strong_disagreement_count = 0
        moderate_disagreement_count = 0
        
        # Debug: sample some utterances to see what we're working with
        sample_texts = []
        texts_processed = 0
        
        for utt in utterances:
            # Safely get text
            try:
                if not hasattr(utt, 'text') or not utt.text:
                    continue
                text_lower = str(utt.text).lower()
                texts_processed += 1
                
                # Collect sample texts for debugging
                if len(sample_texts) < 3:
                    sample_texts.append(text_lower[:200])
            except (AttributeError, TypeError) as e:
                continue
            
            # Count strong agreement (once per utterance)
            found_strong_agree = False
            for pattern in strong_agreement_patterns:
                try:
                    match = re.search(pattern, text_lower)
                    if match:
                        strong_agreement_count += 1
                        found_strong_agree = True
                        break
                except Exception as e:
                    # Debug: if pattern matching fails, log it
                    if texts_processed <= 3:  # Only log for first few
                        print(f"[DEBUG] Pattern match error for '{pattern}': {e}")
                    continue
            
            # Count moderate agreement (only if no strong agreement found)
            if not found_strong_agree:
                for pattern in moderate_agreement_patterns:
                    try:
                        if re.search(pattern, text_lower):
                            moderate_agreement_count += 1
                            break
                    except Exception:
                        continue
            
            # Count strong disagreement (once per utterance)
            found_strong_disagree = False
            for pattern in strong_disagreement_patterns:
                try:
                    if re.search(pattern, text_lower):
                        strong_disagreement_count += 1
                        found_strong_disagree = True
                        break
                except Exception:
                    continue
            
            # Count moderate disagreement (only if no strong disagreement found)
            if not found_strong_disagree:
                for pattern in moderate_disagreement_patterns:
                    try:
                        if re.search(pattern, text_lower):
                            moderate_disagreement_count += 1
                            break
                    except Exception:
                        continue
        
        # Weighted counts (strong markers count more)
        total_agreement = strong_agreement_count * 2 + moderate_agreement_count
        total_disagreement = strong_disagreement_count * 2 + moderate_disagreement_count
        
        print(f"[DEBUG] Counts - Strong agree: {strong_agreement_count}, Moderate agree: {moderate_agreement_count}")
        print(f"[DEBUG] Counts - Strong disagree: {strong_disagreement_count}, Moderate disagree: {moderate_disagreement_count}")
        print(f"[DEBUG] Total agreement: {total_agreement}, Total disagreement: {total_disagreement}")
        
        # Debug output
        if total_agreement == 0 and total_disagreement == 0:
            print(f"[DEBUG] No markers found. Processed {texts_processed} utterances from {len(utterances)} total.")
            if sample_texts:
                print(f"[DEBUG] Sample texts (first 200 chars each):")
                for i, text in enumerate(sample_texts, 1):
                    print(f"  {i}: {text}...")
            # Try a simple test - check if common words exist at all
            if texts_processed > 0:
                test_text = " ".join(sample_texts[:3]) if sample_texts else ""
                if test_text:
                    # Check if basic words exist
                    has_yes = 'yes' in test_text
                    has_but = 'but' in test_text
                    has_agree = 'agree' in test_text
                    print(f"[DEBUG] Simple word check - 'yes': {has_yes}, 'but': {has_but}, 'agree': {has_agree}")
                    # Try a simple regex test
                    test_match = re.search(r'\byes\b', test_text)
                    print(f"[DEBUG] Regex test for '\\byes\\b': {test_match is not None}")
        
        # Handle edge cases
        if total_agreement == 0 and total_disagreement == 0:
            # No clear markers found - default to neutral
            return 5.0
        elif total_disagreement == 0:
            # Only agreement found - strong consensus
            return 1.0
        elif total_agreement == 0:
            # Only disagreement found - strong divergence
            return 10.0
        
        # Calculate ratio and score
        agreement_ratio = total_agreement / total_disagreement
        
        print(f"[DEBUG] Agreement ratio: {agreement_ratio:.3f}")
        
        # Map ratio to 1-10 scale using a smoother continuous function
        # High ratio (lots of agreement) -> low score (consensus)
        # Low ratio (lots of disagreement) -> high score (divergence)
        # Ratio = 1.0 (balanced) -> score = 5.0
        
        # Use logarithmic scaling for smoother transitions
        # When ratio = 1.0, log(1.0) = 0, so score = 5.0
        # When ratio -> infinity (all agreement), score -> 1.0
        # When ratio -> 0 (all disagreement), score -> 10.0
        
        import math
        
        if agreement_ratio > 0:
            # Use log scale: log(ratio) maps to score
            # log(1) = 0 -> score 5.0
            # log(10) ≈ 2.3 -> score ~1.0 (strong consensus)
            # log(0.1) ≈ -2.3 -> score ~10.0 (strong divergence)
            
            log_ratio = math.log(agreement_ratio)
            
            # Map log_ratio from [-2.3, 2.3] to [10, 1]
            # When log_ratio = 0 (ratio = 1.0), score = 5.0
            # When log_ratio = 2.3 (ratio ≈ 10), score ≈ 1.0
            # When log_ratio = -2.3 (ratio ≈ 0.1), score ≈ 10.0
            
            # Linear mapping: log_ratio of 0 -> 5.0, log_ratio of 2.3 -> 1.0, log_ratio of -2.3 -> 10.0
            if log_ratio >= 0:
                # Consensus range: log_ratio [0, 2.3] -> score [5.0, 1.0]
                score = 5.0 - (log_ratio / 2.3) * 4.0
            else:
                # Divergence range: log_ratio [-2.3, 0] -> score [10.0, 5.0]
                score = 5.0 - (log_ratio / 2.3) * 5.0
        else:
            # Shouldn't happen (we check for total_disagreement == 0 above)
            score = 10.0
        
        score = max(1.0, min(10.0, score))
        print(f"[DEBUG] Computed score: {score:.3f}")
        
        return score
        
    except AttributeError as e:
        # Handle missing attributes gracefully
        print(f"[DEBUG] AttributeError in compute_conclusiveness_score: {e}")
        return 5.0
    except Exception as e:
        # Log the actual error for debugging (but don't fail)
        print(f"[DEBUG] Exception in compute_conclusiveness_score: {type(e).__name__}: {str(e)}")
        import traceback
        print(f"[DEBUG] Traceback: {traceback.format_exc()}")
        return 5.0

def compute_speaker_identity_score(corpus: Corpus) -> float:
    """Speaker Identity: Similarity (1) vs. Diversity (10)"""
    try:
        speakers = {}
        
        for utt in corpus.iter_utterances():
            speaker_id = utt.speaker.id
            if speaker_id not in speakers:
                speakers[speaker_id] = {"words": set()}
            
            words = set(re.findall(r'\b\w+\b', utt.text.lower()))
            speakers[speaker_id]["words"].update(words)
        
        if len(speakers) < 2:
            return 5.0
        
        speaker_list = list(speakers.keys())
        overlaps = []
        
        for i in range(len(speaker_list)):
            for j in range(i + 1, len(speaker_list)):
                words_i = speakers[speaker_list[i]]["words"]
                words_j = speakers[speaker_list[j]]["words"]
                
                if len(words_i) == 0 or len(words_j) == 0:
                    continue
                
                overlap = len(words_i & words_j) / len(words_i | words_j)
                overlaps.append(overlap)
        
        if not overlaps:
            return 5.0
        
        avg_overlap = np.mean(overlaps)
        score = 10 - (avg_overlap * 9)
        return max(1, min(10, score))
    except Exception as e:
        print(f"Error computing speaker identity score: {e}")
        return 5.0

def compute_speaker_fluidity_score(corpus: Corpus, window_size: int = 20) -> float:
    """Speaker Fluidity: Malleability (1) vs. Consistency (10)"""
    try:
        speaker_utterances = defaultdict(list)
        
        for utt in corpus.iter_utterances():
            speaker_utterances[utt.speaker.id].append({
                "text": utt.text,
                "timestamp": utt.meta.get("timestamp", 0)
            })
        
        if len(speaker_utterances) == 0:
            return 5.0
        
        # Use adaptive window size: at least 4 utterances (2 per half), or use the specified window_size
        # Find the minimum utterances any speaker has, and use that to set a reasonable threshold
        min_utterances = min(len(utts) for utts in speaker_utterances.values()) if speaker_utterances else 0
        # Use at least 4, but prefer the specified window_size if speakers have enough utterances
        adaptive_window = max(4, min(window_size, min_utterances)) if min_utterances >= 4 else 4
        
        consistency_scores = []
        
        for speaker_id, utts in speaker_utterances.items():
            # Require at least 4 utterances to split into two halves
            if len(utts) < 4:
                continue
            
            utts_sorted = sorted(utts, key=lambda x: x["timestamp"])
            mid_point = len(utts_sorted) // 2
            
            first_half_words = set()
            second_half_words = set()
            
            for utt in utts_sorted[:mid_point]:
                words = set(re.findall(r'\b\w+\b', utt["text"].lower()))
                first_half_words.update(words)
            
            for utt in utts_sorted[mid_point:]:
                words = set(re.findall(r'\b\w+\b', utt["text"].lower()))
                second_half_words.update(words)
            
            if len(first_half_words) == 0 or len(second_half_words) == 0:
                continue
            
            overlap = len(first_half_words & second_half_words)
            union = len(first_half_words | second_half_words)
            similarity = overlap / union if union > 0 else 0
            consistency_scores.append(similarity)
        
        if not consistency_scores:
            return 5.0
        
        avg_consistency = np.mean(consistency_scores)
        score = 1 + (avg_consistency * 9)
        return max(1, min(10, score))
    except Exception as e:
        print(f"Error computing speaker fluidity score: {e}")
        return 5.0

def evaluate_conversation(conversation_history: List[Dict]) -> Dict[str, float]:
    """Compute all 4 evaluation metrics using ConvoKit."""
    corpus = conversation_to_corpus(conversation_history)
    
    return {
        "dynamic": compute_dynamic_score(corpus),
        "conclusiveness": compute_conclusiveness_score(corpus),
        "speaker_identity": compute_speaker_identity_score(corpus),
        "speaker_fluidity": compute_speaker_fluidity_score(corpus)
    }


## Running Experiments

Let's run conversations with 2 and 3 speakers, then analyze the implicit personas that emerge.


In [28]:
# Run 2-speaker conversation
print("Running 2-speaker conversation with implicit personas...")
conv_2speaker = run_conversation_implicit_personas(iterations=50, participant_count=2)
metrics_2speaker = evaluate_conversation(conv_2speaker)

print("\\n=== 2-Speaker Results ===")
print(f"Dynamic (Collaborative ↔ Competitive): {metrics_2speaker['dynamic']:.2f}/10")
print(f"Conclusiveness (Consensus ↔ Divergence): {metrics_2speaker['conclusiveness']:.2f}/10")
print(f"Speaker Identity (Similarity ↔ Diversity): {metrics_2speaker['speaker_identity']:.2f}/10")
print(f"Speaker Fluidity (Malleability ↔ Consistency): {metrics_2speaker['speaker_fluidity']:.2f}/10")


Running 2-speaker conversation with implicit personas...


Running 2-speaker conversation: 100%|██████████| 50/50 [05:08<00:00,  6.17s/it]


\n=== 2-Speaker Results ===
Dynamic (Collaborative ↔ Competitive): 5.32/10
Conclusiveness (Consensus ↔ Divergence): 5.00/10
Speaker Identity (Similarity ↔ Diversity): 5.25/10
Speaker Fluidity (Malleability ↔ Consistency): 4.65/10


In [34]:
metrics_2speaker = evaluate_conversation(conv_2speaker)

print("\\n=== 2-Speaker Results ===")
print(f"Dynamic (Collaborative ↔ Competitive): {metrics_2speaker['dynamic']:.2f}/10")
print(f"Conclusiveness (Consensus ↔ Divergence): {metrics_2speaker['conclusiveness']:.2f}/10")
print(f"Speaker Identity (Similarity ↔ Diversity): {metrics_2speaker['speaker_identity']:.2f}/10")
print(f"Speaker Fluidity (Malleability ↔ Consistency): {metrics_2speaker['speaker_fluidity']:.2f}/10")

[DEBUG] compute_conclusiveness_score called
[DEBUG] Corpus type: <class 'convokit.model.corpus.Corpus'>
[DEBUG] Retrieved 46 utterances
[DEBUG] Counts - Strong agree: 13, Moderate agree: 14
[DEBUG] Counts - Strong disagree: 2, Moderate disagree: 36
[DEBUG] Total agreement: 40, Total disagreement: 40
[DEBUG] Agreement ratio: 1.000
[DEBUG] Computed score: 5.000
\n=== 2-Speaker Results ===
Dynamic (Collaborative ↔ Competitive): 5.32/10
Conclusiveness (Consensus ↔ Divergence): 5.00/10
Speaker Identity (Similarity ↔ Diversity): 5.25/10
Speaker Fluidity (Malleability ↔ Consistency): 4.65/10


In [ ]:
# Run 3-speaker conversation
print("\\nRunning 3-speaker conversation with implicit personas...")
conv_3speaker = run_conversation_implicit_personas(iterations=33, participant_count=3)

\nRunning 3-speaker conversation with implicit personas...


Running 3-speaker conversation: 100%|██████████| 33/33 [07:43<00:00, 14.05s/it]


\n=== 3-Speaker Results ===
Dynamic (Collaborative ↔ Competitive): 5.43/10
Conclusiveness (Consensus ↔ Divergence): 5.00/10
Speaker Identity (Similarity ↔ Diversity): 5.26/10
Speaker Fluidity (Malleability ↔ Consistency): 4.82/10


In [35]:
metrics_3speaker = evaluate_conversation(conv_3speaker)

print("\\n=== 3-Speaker Results ===")
print(f"Dynamic (Collaborative ↔ Competitive): {metrics_3speaker['dynamic']:.2f}/10")
print(f"Conclusiveness (Consensus ↔ Divergence): {metrics_3speaker['conclusiveness']:.2f}/10")
print(f"Speaker Identity (Similarity ↔ Diversity): {metrics_3speaker['speaker_identity']:.2f}/10")
print(f"Speaker Fluidity (Malleability ↔ Consistency): {metrics_3speaker['speaker_fluidity']:.2f}/10")

[DEBUG] compute_conclusiveness_score called
[DEBUG] Corpus type: <class 'convokit.model.corpus.Corpus'>
[DEBUG] Retrieved 70 utterances
[DEBUG] Counts - Strong agree: 18, Moderate agree: 14
[DEBUG] Counts - Strong disagree: 3, Moderate disagree: 60
[DEBUG] Total agreement: 50, Total disagreement: 66
[DEBUG] Agreement ratio: 0.758
[DEBUG] Computed score: 5.604
\n=== 3-Speaker Results ===
Dynamic (Collaborative ↔ Competitive): 5.43/10
Conclusiveness (Consensus ↔ Divergence): 5.60/10
Speaker Identity (Similarity ↔ Diversity): 5.26/10
Speaker Fluidity (Malleability ↔ Consistency): 4.82/10


## Analyzing Implicit Personas

Let's look at the actual messages to see what personas emerged.


In [36]:
# Show first messages from each speaker (their initial personas)
print("=== Initial Personas (First Messages) ===")
for msg in conv_2speaker[:2]:
    speaker = msg.get("name", "unknown")
    content = msg.get("content", "")
    print(f"\\n{speaker}:")
    print(content[:200] + "..." if len(content) > 200 else content)

# Show later messages to see if personas persisted
print("\\n\\n=== Later Messages (Persona Persistence) ===")
for msg in conv_2speaker[-5:]:
    speaker = msg.get("name", "unknown")
    content = msg.get("content", "")
    print(f"\\n{speaker}: {content[:150]}...")


=== Initial Personas (First Messages) ===
\nspeaker_1:
Thank you! I'm glad to be here. What's on everyone's mind today?
\nspeaker_2:
Thank you! I'm glad to be here. What's on the agenda for our discussion today?
\n\n=== Later Messages (Persona Persistence) ===
\nspeaker_2: Reflecting on the collective insights and dynamic discussions we've had, it's truly motivating to witness our commitment to integrating AI ethically a...
\nspeaker_1: Speaker_2, I truly appreciate your perspectives and the enthusiasm you've brought to our discussion about the ethical integration of AI. It's been inc...
\nspeaker_2: Reflecting on the depth and breadth of our ongoing discussion, I find it incredibly encouraging to see us, as a group, united by a shared commitment t...
\nspeaker_1: Our discussion on integrating AI ethically across various sectors has generated quite a wealth of insights, each shedding light on the shared commitme...
\nspeaker_2: It's been insightful to engage in this ongoing dialogue ab

## Comparison with Explicit Personas

### Key Observations

1. **Natural Differentiation**: Even without explicit prompts, LLMs develop distinct communication styles
2. **Speaker Identity Score**: Measures how different speakers are from each other
   - High score (8-10): Speakers are very different (diverse personas)
   - Low score (1-3): Speakers are similar (convergent personas)
3. **Speaker Fluidity**: Measures consistency over time
   - High score (8-10): Speakers maintain consistent style
   - Low score (1-3): Speakers change style frequently

### Implicit vs Explicit Personas

**Implicit Personas (this post)**:
- Develop naturally during conversation
- May be less distinct initially
- Can evolve more freely

**Explicit Personas (post 007)**:
- Guided by initial prompts
- More distinct from the start
- May be more stable but less flexible

## Summary

LLMs naturally develop implicit personas even without explicit guidance:

- **Differentiation occurs**: Speaker Identity scores show distinct personas emerge
- **Consistency varies**: Some speakers maintain style, others adapt
- **Comparable to explicit**: Implicit personas can be as distinct as explicit ones

This suggests that:
1. LLMs have inherent tendencies toward persona development
2. Conversation context shapes persona emergence
3. Explicit prompts may guide but don't create personas from scratch

Future work could explore:
- What factors influence implicit persona development?
- How do implicit personas compare to explicit ones in long conversations?
- Can we predict which implicit personas will emerge?
